In [1]:
import numpy
import pandas
import matplotlib.pyplot as plt
import ESN_Modules as ESN
from scipy import linalg
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
def test_parameters(X, train_dataset, test_dataset, train_length, test_length, transiente, N, K, scale, radius, sparsity, leak):
    numpy.random.seed(X)
    # PESOS DE ENTRADA
    W_input = ESN.input_weights(N = N, K = K, input_scaling = scale)
    # PESOS DE RESERVATORIO
    W_reservoir = ESN.reservoir_weights(N = N, spectral_radius = radius, sparsity = sparsity)
    
    # MATRIZ DE ESTADOS (TRAINNING)
    state_matrix = ESN.harvesting_states(state_matrix = ESN.create_states(dim = 1 + K + N, patterns = train_length - transiente), 
                                         input_patterns = train_dataset, 
                                         win = W_input, 
                                         wres = W_reservoir, 
                                         initial_state = numpy.zeros((N, 1)),
                                         leaking = leak, 
                                         transiente = transiente)

    # PESOS DE SAÍDA
    W_output = ESN.ridge_regression(states = state_matrix, target = train_target)
    
    train_predict = numpy.dot(W_output, state_matrix)
    
    # O ESTADO INICIAL É O ÚLTIMO ESTADO DO TREINAMENTO
    initial_state = ESN.reshape(state_matrix[K + 1::,-1])

    # MATRIX DE ESTADOS (TESTING)
    state_matrix = ESN.harvesting_states(state_matrix = ESN.create_states(dim = 1 + K + N, patterns = test_length), 
                                         input_patterns = test_dataset, 
                                         win = W_input, 
                                         wres = W_reservoir, 
                                         initial_state = initial_state,
                                         leaking = leak, 
                                         transiente = 0)
    # ERRO DE PREDIÇÃO
    return(train_predict, numpy.dot(W_output, state_matrix))

In [3]:
# TAMANHO DE JANELA DE ENTRADA (INPUT)
len_window_input = 1
# TAMANHO DE JANELA DE SAÍDA (OUTPUT)
len_window_output = 1

# PASSOS AHEAD QUE DESEJAMOS PREVER
steps = 1

# TAMANHO DA JANELA DE TRAINNING
train_length = 3500
# TAMANHO DA JANELA DE TESTING
test_length = 1500

# TRANSIENTE, VALORES A SEREM DESCARTADOS
transiente = int(train_length*0.1)

# NÚMERO DE NEURÔNIOS NA CAMADA OCULTA, INT, N > 0
neurons = numpy.linspace(start = 20, stop = 500, num = 25).astype(int)

# NÚMERO DE RODADAS
rounds = 10

# ESPARSIDADE DA MATRIZ, FLOAT, NO CONJUNTO [0, 1]
sparsities = [0.75]
# RAIO ESPECTRAL DESEJADO, FLOAT, NO CONJUNTO (0, INF)
spectral_radius = [0.7]
# TAXA DE VAZAMENTO ESPECTRAL, FLOAT, NO CONJUNTO (0, 1]
leaking_rates = [0.95]
# ESCALA DOS DADOS DE ENTRADA, FLOAT, NO CONJUNTO (0, INF)
input_scalings = [0.3]

In [4]:
#trainning_data = pandas.read_csv("ParametrosTreino_NoiseMackey22.txt")
#trainning_data = trainning_data[(trainning_data.LEAK == leaking_rates[0]) & (trainning_data.SCALE == input_scalings[0]) & (trainning_data.SPARSITY == sparsities[0]) & (trainning_data.RADIUS == spectral_radius[0])]
#trainning_data

In [5]:
# DEFINE UMA SEMENTE DO GERADOR PSEUDO-ALEATORIO
numpy.random.seed(42)

cols = ["ROUND", "WINDOW", "NEURONS", "LEAK", "SPARSITY", "SCALE", "RADIUS", "MSE", "RMSE", "MAPE", 
        "MSE(X)", "RMSE(X)", "MAPE(X)", "MSE(Y)", "RMSE(Y)", "MAPE(Y)", "MSE(Z)", "RMSE(Z)", "MAPE(Z)",
        "FRAMEWORK"]

sample = len(neurons) * rounds

noise_weight = 0.0025

#architectures = trainning_data["FRAMEWORK"].astype(int).tolist()
# CORRIGIR DEPOIS
architectures = numpy.random.randint(low = 1, high = 10000, size = rounds)


train_simulation = pandas.DataFrame(numpy.nan, columns = cols, 
                                    index = numpy.linspace(start = 0, stop = sample - 1, num = sample).astype(int))
valid_simulation = pandas.DataFrame(numpy.nan, columns = cols, 
                                    index = numpy.linspace(start = 0, stop = sample - 1, num = sample).astype(int))
i = 0

#eval_model = train_target[0].T

for rnd in range(rounds):
    
    # INPUT DO SISTEMA
    #numpy.random.seed(architectures[rnd])
    input_dataset, K = ESN.set_window(name = "series_mackey22_norm.txt", window_len = len_window_input, begin = 0)

    # OUTPUT ESPERADO
    output_dataset, L = ESN.set_window(name = "series_mackey22_norm.txt", window_len = len_window_output, begin = len_window_input)
    #output_dataset += numpy.random.normal(0, noise_weight, output_dataset.shape)

    # DATASET DE TREINO -> [(X0), (X1), (X2), ..., (Xjanela_treino-1)]
    train_dataset = input_dataset[0 : train_length, :]
    # DATASET DE TREINO (TARGET) -> [(Ysteps+transiente), (Ysteps+transiente+1), ..., (Ysteps+janela_treino-1)]
    train_target = output_dataset[None, steps + transiente - 1 : train_length + steps - 1, :]

    # DATASET DE TESTE -> [(Xjanela_treino), (Xjanela_treino+1), (Xjanela_treino+2), ..., (Xjanela_treino+janela_teste-1)]
    test_dataset = input_dataset[train_length : train_length + test_length, :]
    # DATASET DE TESTE (TARGET) -> [(Ysteps+janela_treino+janela_teste), ..., (Ysteps+janela_treino+janela_teste-1)]
    test_target = output_dataset[train_length + steps - 1 : train_length + test_length + steps - 1, :]
    
    for N in neurons:
        for leak in leaking_rates:
            for sparsity in sparsities:
                for scale in input_scalings:
                    for radius in spectral_radius:
                        train_predict, predict = test_parameters(
                            X = architectures[rnd], 
                            train_dataset = train_dataset, 
                            test_dataset = test_dataset, 
                            train_length = train_length, 
                            test_length = test_length, 
                            transiente = transiente, 
                            N = N, 
                            K = K, 
                            scale = scale, 
                            radius = radius, 
                            sparsity = sparsity, 
                            leak = leak
                        )
                        #print(train_target[0].shape)
                        #print(test_target.shape)
                        #print(train_predict.shape)
                        #print(predict.shape)
                        train_simulation.iloc[i,:] = [rnd, len_window_input, N, leak, sparsity, scale, radius] + ESN.model_metrics(train_target[0].T, train_predict) + [architectures[rnd]]
                        valid_simulation.iloc[i,:] = [rnd, len_window_input, N, leak, sparsity, scale, radius] + ESN.model_metrics(test_target.T, predict) + [architectures[rnd]]
                        
                        i += 1

In [6]:
train_simulation

,ROUND,WINDOW,NEURONS,LEAK,SPARSITY,SCALE,RADIUS,MSE,RMSE,MAPE,MSE(X),RMSE(X),MAPE(X),MSE(Y),RMSE(Y),MAPE(Y),MSE(Z),RMSE(Z),MAPE(Z),FRAMEWORK
0,0.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000062,0.005420,0.020235,0.000062,0.005420,0.020235,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
1,0.0,1.0,40.0,0.95,0.75,0.3,0.7,0.000045,0.004492,0.016402,0.000045,0.004492,0.016402,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
2,0.0,1.0,60.0,0.95,0.75,0.3,0.7,0.000044,0.004427,0.016836,0.000044,0.004427,0.016836,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
3,0.0,1.0,80.0,0.95,0.75,0.3,0.7,0.000039,0.004113,0.015936,0.000039,0.004113,0.015936,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
4,0.0,1.0,100.0,0.95,0.75,0.3,0.7,0.000039,0.004082,0.014874,0.000039,0.004082,0.014874,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,9.0,1.0,420.0,0.95,0.75,0.3,0.7,0.000034,0.003780,0.015471,0.000034,0.003780,0.015471,NaN,NaN,NaN,NaN,NaN,NaN,8323.0
246,9.0,1.0,440.0,0.95,0.75,0.3,0.7,0.000033,0.003744,0.015267,0.000033,0.003744,0.015267,NaN,NaN,NaN,NaN,NaN,NaN,8323.0
247,9.0,1.0,460.0,0.95,0.75,0.3,0.7,0.000034,0.003797,0.016267,0.000034,0.003797,0.016267,NaN,NaN,NaN,NaN,NaN,NaN,8323.0
248,9.0,1.0,480.0,0.95,0.75,0.3,0.7,0.000033,0.003759,0.015352,0.000033,0.003759,0.015352,NaN,NaN,NaN,NaN,NaN,NaN,8323.0


In [7]:
valid_simulation

,ROUND,WINDOW,NEURONS,LEAK,SPARSITY,SCALE,RADIUS,MSE,RMSE,MAPE,MSE(X),RMSE(X),MAPE(X),MSE(Y),RMSE(Y),MAPE(Y),MSE(Z),RMSE(Z),MAPE(Z),FRAMEWORK
0,0.0,1.0,20.0,0.95,0.75,0.3,0.7,0.000071,0.005811,0.030547,0.000071,0.005811,0.030547,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
1,0.0,1.0,40.0,0.95,0.75,0.3,0.7,0.000052,0.004853,0.022652,0.000052,0.004853,0.022652,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
2,0.0,1.0,60.0,0.95,0.75,0.3,0.7,0.000051,0.004808,0.021531,0.000051,0.004808,0.021531,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
3,0.0,1.0,80.0,0.95,0.75,0.3,0.7,0.000046,0.004539,0.020655,0.000046,0.004539,0.020655,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
4,0.0,1.0,100.0,0.95,0.75,0.3,0.7,0.000046,0.004462,0.018570,0.000046,0.004462,0.018570,NaN,NaN,NaN,NaN,NaN,NaN,7271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,9.0,1.0,420.0,0.95,0.75,0.3,0.7,0.000041,0.004179,0.016892,0.000041,0.004179,0.016892,NaN,NaN,NaN,NaN,NaN,NaN,8323.0
246,9.0,1.0,440.0,0.95,0.75,0.3,0.7,0.000040,0.004136,0.016887,0.000040,0.004136,0.016887,NaN,NaN,NaN,NaN,NaN,NaN,8323.0
247,9.0,1.0,460.0,0.95,0.75,0.3,0.7,0.000041,0.004189,0.017710,0.000041,0.004189,0.017710,NaN,NaN,NaN,NaN,NaN,NaN,8323.0
248,9.0,1.0,480.0,0.95,0.75,0.3,0.7,0.000041,0.004155,0.017072,0.000041,0.004155,0.017072,NaN,NaN,NaN,NaN,NaN,NaN,8323.0


In [9]:
train_simulation.to_csv("ControleTrainMackey22.txt", index = False)
valid_simulation.to_csv("ControleValidMackey22.txt", index = False)